In [1]:
import numpy as np
import cv2
from tensorflow import keras
from scipy.spatial import distance as dist
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [2]:
def mask_label(pred):
    if(pred>0.5):
        return 'NO MASK'
    else:
        return 'MASK'
def color_label(pred):
    if(pred>0.5):
        return (0,0,255)
    else:
        return (0,255,0)
# dist_label = {1:(0,255,0),0:(255,0,0)}

In [4]:
model = keras.models.load_model('MaskXception.h5')

In [5]:
#cap = cv2.VideoCapture('sample.mp4')
cap = cv2.VideoCapture(0)

In [ ]:
s=80
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.FONT_HERSHEY_SIMPLEX
    )
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        crop=new_img[y:y+h,x:x+w]
        if(type(faces)!=tuple):
            img = cv2.resize(crop,(s,s))
            image = np.reshape(img,[1,s,s,3])/255.0
            predict=model.predict(image)
            cv2.rectangle(frame, (x, y), (x+w, y+h),color_label(predict), 2)
            cv2.putText(frame,mask_label(predict),(x-10, y-10),cv2.FONT_HERSHEY_PLAIN,2,color_label(predict),2)
    x_mid=[]
    y_mid=[]
    x_mid1=[]
    y_mid1=[]
    if(len(faces)>0):
        for i in range(0,len(faces)):
            x_mid.append(int(faces[i][0]+0.5*faces[i][2]))
            y_mid.append(int(faces[i][1]+0.5*faces[i][3]))
            x_mid1.append(int(faces[i][0]+0.5*faces[i][2]))
            y_mid1.append(int(faces[i][1]+0.5*faces[i][3]))
            cv2.circle(frame, (x_mid[i],y_mid[i]), 3 , [255,0,0] , -1)
        if(len(faces)>1):
            d=int(dist.euclidean((x_mid.pop(),y_mid.pop()),(x_mid.pop(),y_mid.pop())))
            print(d)
            if(d<250):
                cv2.putText(frame, "You are in Danger", (20, 20), cv2.FONT_HERSHEY_PLAIN,2,[0,0,255] ,2)
                cv2.line(frame,(x_mid1.pop(),y_mid1.pop()),(x_mid1.pop(),y_mid1.pop()),[0,0,255], 2)

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

103
399
344
